In [1]:
import numpy as np
import json
import tensorflow as tf
import os
import sys

import time
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import gym

import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(gpus[0], 
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=256)])
  except RuntimeError as e:
    print(e)
    
    
print("GPUs: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.config.experimental.list_logical_devices('GPU')

import tf_agents
from tf_agents.environments import py_environment, parallel_py_environment, batched_py_environment
from tf_agents.environments import tf_environment
from tf_agents.environments import tf_py_environment
from tf_agents.environments import utils
from tf_agents.specs import array_spec
from tf_agents.environments import wrappers
from tf_agents.environments import suite_gym
from tf_agents.trajectories import time_step as ts
from tf_agents.networks.q_network import QNetwork
from tf_agents.agents.dqn.dqn_agent import DqnAgent
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.metrics import tf_metrics
#from tf_agents.drivers.dynamic_step_driver import DynamicStepDriver
from tf_agents.drivers.dynamic_episode_driver import DynamicEpisodeDriver
from tf_agents.policies.random_tf_policy import RandomTFPolicy
from tf_agents.utils.common import function, element_wise_squared_loss
from tf_agents.eval.metric_utils import log_metrics
from tf_agents.policies import policy_saver
import logging

import tensorflow.keras as keras

tf.compat.v1.enable_v2_behavior()
import time
import json
import datetime
import copy

from scan_gym import envs
seed=42
tf.random.set_seed(seed)
np.random.seed(seed)

#import imp
#import test as nntools
#imp.reload(nntools)
import csv
import json
import copy

GPUs:  1


In [2]:
data_template = {'actions':[],'images':[],'position_bias':[],'return':[],'gt_ratio':[],'empty_vox':[],
                 'ind_vox':[],'solid_vox':[],'gt_ratio_mean':0.0,'gt_ratio_std':0}

In [3]:
def run_episode(env,tf_env,policy):
    state = tf_env.reset()
    time_steps = 40
    actions = []
    for i in range(1,time_steps):
        action_step = policy.action(state)
        actions.append(int(action_step.action.numpy()[0]))
        state = tf_env.step(action_step.action)
        if state.is_last():
            break
    return actions, env.kept_images, env.position_bias, env.total_reward , env.last_gt_ratio , env.h

In [4]:
policy_name ='20211008-164658'
policies_path = '/home/pico/uni/romi/desktop_scanner_cube_repo/scanner_cube'
plant_models_path = '/home/pico/uni/romi/scanner_cube/arabidopsis_im_bigger'
plant_models = [1,3,25,39,41,6,9,11,14,22,0,20,28,36,48,201,202,203,204,205]

policy = tf.compat.v2.saved_model.load(os.path.join(policies_path,policy_name))

In [5]:
collected_data = {}

for p in plant_models:
    #load environment with selected plant model
    env = suite_gym.load('ScannerEnv-v1',
    gym_kwargs={'dataset_path':os.path.join(plant_models_path,str(p).zfill(3)),'gt_mode':True,'init_pos_inc_rst':True}) 
    tf_env = tf_py_environment.TFPyEnvironment( env )
    #run 180 times (rotating model one step each time)
    data_holder = copy.deepcopy(data_template)
    
    for i in range(180):
        actions, images, position_bias, t_return, gt_ratio , histogram = run_episode(env,tf_env,policy)
        data_holder['actions'].append(actions)
        data_holder['images'].append(images)
        data_holder['position_bias'].append(position_bias)
        data_holder['return'].append(t_return)
        data_holder['gt_ratio'].append(gt_ratio)
        data_holder['empty_vox'].append(histogram[0])
        data_holder['ind_vox'].append(histogram[1])
        data_holder['solid_vox'].append(histogram[2])
        print("\rplant-{} {}     ".format(str(p).zfill(3), i), end="")
    
    data_holder['gt_ratio_mean'] = float(np.mean(data_holder['gt_ratio']))
    data_holder['gt_ratio_std'] = float(np.std(data_holder['gt_ratio']))
    collected_data[str(p).zfill(3)] = copy.deepcopy(data_holder)
    print("\rplant-{} gt_mean {}   gt_std {}            ".format(str(p).zfill(3),data_holder['gt_ratio_mean'],data_holder['gt_ratio_std']))

plant-001 6     

KeyboardInterrupt: 

In [ ]:
dest_path = '/home/pico/uni/romi/desktop_scanner_cube_repo/scanner_cube'
f = open(os.path.join(dest_path,"test_" + policy_name+ ".json"),'w')
json.dump(collected_data, f)
f.close()